In [1]:
#!pip install ipynb

In [2]:
import numpy as np
import threading
import random
import time
import numpy as np
import threading
import random
from protocol_tests import test_all
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


import sys
try:
    del sys.modules['ipynb.fs.full.weak_global_coin']
except KeyError:
    pass

import ipynb.fs.full.weak_global_coin as QCF


try:
    del sys.modules['globals']
except KeyError:
    pass
from globals import *

In [3]:
broadcasted_messages = []
first_to_decide = None

QUESTION_MARK = "?"
WAITING_MESSAGE = "waiting"
HALF_PLUS_ONE = int(np.floor(n/2)) + 1

broadcasting_lock = threading.Lock()
decision_lock = threading.Lock()

timeout = 10

In [4]:
class Process:
    def __init__(self, id, input_val, faulty = False) -> None:
        self.id = id
        self.input_val = input_val
        self.round_messages = {}
        self.output = None
        self.decision_epoch = None
        self.faulty = faulty
    def __str__(self):
        return f"id: {self.id} | round_messages: {self.round_messages}"
    
class BroadcastMessage(Message):
    def __init__(self, process_id, receivers, epoch, round, message) -> None:
        super().__init__(process_id, receivers)
        self.epoch = epoch
        self.round = round
        self.message = message
        self.read = [False for number in range(n)]
    def __str__(self):
        return f"sender: {self.sender} | epoch: {self.epoch} | round: {self.round} | message: {self.message}"

In [5]:
def broadcast(process_id, epoch, round, message):
    new_msg = BroadcastMessage(process_id, list(range(n)), epoch, round, message)
    broadcasting_lock.acquire()
    broadcasted_messages.append(new_msg)
    broadcasting_lock.release()

def waiting_condition(num_received_messages, round):
    if round == 1 or round == 2:
        actual_alive_processes = [1 for pr in processes if pr.faulty == False].count(1)
        print("Non-faulty processes:", [pr.id for pr in processes if pr.faulty == False])
        return num_received_messages < actual_alive_processes
    elif round == 3:
        return num_received_messages < MAX_ALIVE_PROCESSES

def receive(process, epoch, round, required_val=None):
    num_received_messages = 0
    start_time = time.time()
    
    while waiting_condition(num_received_messages, round):
        if time.time() - start_time > timeout:
            print(f"[Process {process.id}] Timeout in round {round}")
            break

        print(f"[Process {process.id}] waiting... got {num_received_messages}")
        broadcasting_lock.acquire()
        for msg in broadcasted_messages:
            if msg.epoch == epoch and msg.round == round and process.id in msg.receivers and not msg.read[process.id]:
                if round == 3:
                    assert(msg.message == required_val)
                existing_count = process.round_messages.get(msg.message, 0)
                existing_count += 1
                process.round_messages.update({msg.message:existing_count})            

                num_received_messages += 1
                msg.read[process.id] = True
        broadcasting_lock.release()

def get_majority_value(process):
    for value, count in process.round_messages.items():
        if count >= HALF_PLUS_ONE:
            return value
    return QUESTION_MARK

def get_most_frequent_val(process):
    most_frequent_val = max(process.round_messages, key=process.round_messages.get)
    
    if most_frequent_val == QUESTION_MARK:
        process.round_messages.pop(most_frequent_val)
        most_frequent_val = None
        if process.round_messages:          # if the dictionary is not empty after deleting the (first found) most frequent value
            most_frequent_val = max(process.round_messages, key=process.round_messages.get)
    
    answer = most_frequent_val
    number = process.round_messages.get(most_frequent_val, 0)
    return answer, number

In [6]:
def agreement(process):
    current = process.input_val
    next = False
    epoch = 0
    while True:
        epoch += 1
        
        broadcast(process.id, epoch, 1, current)
        if not next:
            receive(process, epoch, 1)        
            current = get_majority_value(process)
        process.round_messages.clear()                  # needed so that round_messages can be reused for the counts of the next round

        broadcast(process.id, epoch, 2, current)
        if not next:
            receive(process, epoch, 2)
            answer, number = get_most_frequent_val(process)
        process.round_messages.clear()

        broadcast(process.id, epoch, 3, WAITING_MESSAGE)
        if not next:
            receive(process, epoch, 3, WAITING_MESSAGE)
        process.round_messages.clear()

        coin = QCF.quantum_coin_flip(processes, process, epoch)

        if next: 
            break
        
        if number >= HALF_PLUS_ONE:
            current = answer
            next = True
            process.decision_epoch = epoch
            decision_lock.acquire()
            global first_to_decide
            if first_to_decide == None:
                first_to_decide = process.id
            decision_lock.release()
        elif number >= 1:
            current = answer
        else:
            current = coin

        if process.faulty:
            print(f"[Faulty Process {process.id}]  Injecting bad data")
            fake_measurement = ''.join(random.choices('01') for _ in range(qb_per_process))
            fake_circuit = Circuit(system=generate_leader_circuit())
            break
    process.output = current
    return current

In [7]:
threads = []
processes = []

faulty_ids = set(random.sample(range(n), t))

for i in range(n):
    input_val = str(random.choice([0, 1]))
    is_faulty = i in faulty_ids
    pr = Process(i, input_val, faulty=is_faulty)
    processes.append(pr)
    thr = threading.Thread(target=agreement, args=((pr,)))
    threads.append(thr)


for thr in threads:    
    thr.start()

for thr in threads:
    thr.join()

print("*******   SOLUTION:   *******")
for pr in processes:
    print("process(", pr.id, ") = ", pr.output, " @epoch: ", pr.decision_epoch, " | input: ", pr.input_val, " | faulty: ", not pr.faulty)

Exception in thread Exception in thread Thread-3 (agreement):
Thread-4 (agreement):
Exception in thread Thread-5 (agreement):
Exception in thread Thread-6 (agreement):
Traceback (most recent call last):
Traceback (most recent call last):
  File "C:\Users\drutt\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\Users\drutt\AppData\Local\Programs\Python\Python313\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\drutt\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\drutt\AppData\Local\Temp\ipykernel_28244\28282946.py", line 25, in agreement
    coin = QCF.quantum_coin_flip(processes, process, epoch)
  File "C:\Users\drutt\Byzantine\weak_global_coin.ipynb", line 258, in quantum_c

Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 0
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0, 2, 3]
[Process 0] waiting... got 1
Non-faulty processes: [0

In [8]:
test_all(processes, first_to_decide, broadcasted_messages)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'